Тест

In [1]:
from sloyka import EmotionRecognizer
from sloyka.src.utils.data_preprocessing.preprocessor import PreprocessorInput
from tqdm import tqdm

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
#загрузка датафрейма
import geopandas as gpd
df = gpd.read_parquet('/Users/test/Documents/code/sloyka/sloyka/sample_data/sample_data_geocoded.parquet')

In [5]:
df.head(2)

,Текст комментария,Street,Numbers,Toponyms,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,message_id,Дата и время
0,Здравствуйте! В Санкт-Петербурге нет Генеральн...,мойка,,None,набережная реки Мойки Санкт-Петербург Россия,[набережная реки Мойки Санкт-Петербург Россия],набережная реки Мойки Санкт-Петербург Россия,набережная реки Мойки,"набережная реки Мойки, Коломна, округ Коломна,...",POINT (30.27929 59.92712),0,2023.01.26 16:32
1,[club143265175|Центральный район Санкт-Петербу...,рубинштейна,,None,улица Рубинштейна Санкт-Петербург Россия,[улица Рубинштейна Санкт-Петербург Россия],улица Рубинштейна Санкт-Петербург Россия,улица Рубинштейна,"улица Рубинштейна, Владимирский округ, Санкт-П...",POINT (30.34356 59.92829),1,2023.01.26 11:55


Препроцессинг данных

In [6]:
df = PreprocessorInput.run(df, column_name='Текст комментария')

2024-07-29 02:31:20.338 | INFO     | sloyka.src.utils.data_preprocessing.preprocessor:preprocess_dataframe:156 - Preprocessing is complete
2024-07-29 02:31:20.340 | INFO     | sloyka.src.utils.data_preprocessing.preprocessor:preprocess_dataframe:157 - Initial df row count: 208
2024-07-29 02:31:20.340 | INFO     | sloyka.src.utils.data_preprocessing.preprocessor:preprocess_dataframe:158 - Processed df row count: 208


Определение эмоции, используя одну модель. По умолчанию стоит модель HuggingFaceModel.Text.Bert_Large, но можно использовать также HuggingFaceModel.Text.Bert_Tiny, HuggingFaceModel.Text.Bert_Base, HuggingFaceModel.Text.Bert_Tiny2

In [7]:
recognizer = EmotionRecognizer()
recognizer.init_base_recognizer()

In [8]:
df['emotion'] = df['Текст комментария'].progress_apply(recognizer.recognize_emotion)

100%|██████████| 208/208 [00:34<00:00,  6.12it/s]


Возможно более точный способ. Берёт среднее из нескольких моделей и возвращает доминирующую эмоцию, если указано average=True, или просто самую максимальную вероятность эмоции из моделей, если verage=False. 

In [17]:
df.sample(3)

,Текст комментария,Street,Numbers,Toponyms,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,message_id,Дата и время,emotion
182,[club143265175|Центральный район Санкт-Петербу...,чернышевский,,проезд,площадь Чернышевского Санкт-Петербург Россия;...,[площадь Чернышевского Санкт-Петербург Россия...,площадь Чернышевского Санкт-Петербург Россия,площадь Чернышевского,"площадь Чернышевского, округ Пулковский мериди...",POINT (30.31702 59.86889),182,2022.11.14 10:56,neutral
47,"ул. Достоевского, д. 5 печальное зрелище помой...",достоевского,,None,улица Достоевского Санкт-Петербург Россия,[улица Достоевского Санкт-Петербург Россия],улица Достоевского Санкт-Петербург Россия,улица Достоевского,"улица Достоевского, Владимирский округ, Санкт-...",POINT (30.34596 59.92294),47,2022.12.19 23:15,sadness
52,"Социалистическая, Загородный до 5 углов, снег,...",загородного,,None,Загородный проспект Санкт-Петербург Россия,[Загородный проспект Санкт-Петербург Россия],Загородный проспект Санкт-Петербург Россия,Загородный проспект,"Загородный проспект, округ Семёновский, Санкт-...",POINT (30.33514 59.92315),52,2022.12.20 10:27,neutral


In [20]:
from IPython.core.display import HTML

HTML(df.drop(columns=['location_options']).explore(column='emotion', tiles='cartodbdark_matter', marker_kwds={'radius':10})._repr_html_())

In [ ]:
# Создание экземпляра класса EmotionRecognizer
emotion_recognizer = EmotionRecognizer()

# Применение функции recognize_average_emotion_from_multiple_models ко всем строкам DataFrame. 
# При average=True подсчёт будет по среднему значению.
# При average=False подсчёт будет по максимальному значению.
df['emotion_average'] = emotion_recognizer.recognize_average_emotion_from_multiple_models(df, 'text', average=True)